# Наш любимый парсер

In [ ]:
!pip install beautifulsoup4

In [4]:
import requests
from bs4 import BeautifulSoup

Делаем ссылку для парсера. 

In [10]:
def get_hrefs(p):
    
    main_part = "https://www.cian.ru/cat.php?"
    suport_part = "deal_type=sale&district%5B0%5D=4&engine_version=2&object_type%5B0%5D=2&offer_type=flat&room1=1&room2=1"

    url = main_part + suport_part + '&p={}'.format(p)
    
    resp = requests.get(url)
    print(resp.status_code)
    soup = BeautifulSoup(resp.content)
    return soup

In [11]:
soup = get_hrefs(5)

200


In [26]:
[i**2 for i in range(2,10,3)]

[4, 25, 64]

In [24]:
s = soup.find_all('div', {'class': '_93444fe79c--card--_yguQ'})
hrefs = [item.find('a', {'class': 'c6e8ba5398--header--1fV2A'}).get("href") 
                     for item in s]
hrefs

['https://www.cian.ru/sale/flat/225743925/',
 'https://www.cian.ru/sale/flat/225677265/',
 'https://www.cian.ru/sale/flat/225599310/',
 'https://www.cian.ru/sale/flat/223137091/',
 'https://www.cian.ru/sale/flat/224314132/',
 'https://www.cian.ru/sale/flat/219798248/',
 'https://www.cian.ru/sale/flat/226081534/',
 'https://www.cian.ru/sale/flat/208216803/',
 'https://www.cian.ru/sale/flat/221411091/',
 'https://www.cian.ru/sale/flat/226078063/',
 'https://www.cian.ru/sale/flat/221411088/',
 'https://www.cian.ru/sale/flat/221411096/',
 'https://www.cian.ru/sale/flat/225454418/',
 'https://www.cian.ru/sale/flat/198960823/',
 'https://www.cian.ru/sale/flat/219686817/',
 'https://www.cian.ru/sale/flat/221411073/',
 'https://www.cian.ru/sale/flat/221411071/',
 'https://www.cian.ru/sale/flat/224866072/',
 'https://www.cian.ru/sale/flat/223320699/',
 'https://www.cian.ru/sale/flat/221411093/',
 'https://www.cian.ru/sale/flat/222781244/',
 'https://www.cian.ru/sale/flat/225639493/',
 'https://

In [18]:
a = 
a

<a class="c6e8ba5398--header--1fV2A" href="https://www.cian.ru/sale/flat/225743925/" target="_blank"><div class="c6e8ba5398--single_title--22TGT">1-комн. кв., 57,9 м², 4/10 этаж</div></a>

In [22]:
a

'https://www.cian.ru/sale/flat/225743925/'

In [24]:
import this

You should consider upgrading via the 'pip install --upgrade pip' command.


In [18]:
url = "https://www.cian.ru/sale/flat/223309049/"
req = requests.get(url)
req

<Response [200]>

In [19]:
req.status_code

200

<span content="5 279 008 ₽" itemprop="price">5 279 008 ₽</span>

In [45]:
import re
re.sub('\D', '', price.text)

'5279008'